# Examples using the U.S. Bureau of Labor Statistics API

## Related Links - Public Data Application Programming Interface (API)
- https://www.bls.gov/bls/api_features.htm
- https://www.bls.gov/developers/api_python.htm#python2
- https://www.bls.gov/developers/api_signature_v2.htm#python2

In [3]:
# write to files
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2014"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()

In [5]:
# print pandas table
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()

headers = {'Content-type': 'application/json'}
data = json.dumps(
        {
        "seriesid": ['LNU02000000'], # (Seas) Employment Level
        "startyear":"1947", 
        "endyear":"2023",
        "registrationkey": os.environ['API_TOKEN_US_BUREAU_OF_LABOR_STATISTICS']
        }
    )
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

json_ts = json_data['Results']['series'][0]['data']
json_ts

df = pd.DataFrame(json_ts)
df

,year,period,periodName,value,footnotes
0,1966,M12,December,73600,[{}]
1,1966,M11,November,73995,[{}]
2,1966,M10,October,73744,[{}]
3,1966,M09,September,73248,[{}]
4,1966,M08,August,74665,[{}]
...,...,...,...,...,...
223,1948,M05,May,57763,[{}]
224,1948,M04,April,57471,[{}]
225,1948,M03,March,56601,[{}]
226,1948,M02,February,56440,[{}]


In [10]:
# get bls data as json
import pandas as pd
import requests
import json
import os

def check_date_range(startyear, endyear):
    year_diff = int(endyear) - int(startyear)
    if year_diff > 20:
        raise Exception('Registered users may request up to 20 years per query.')

def get_bls_data(seriesid, startyear, endyear):
    check_date_range(startyear, endyear)
    filename = 'cache/'+seriesid+'/startyear'+startyear+'_endyear'+endyear+'.json'
    if os.path.isfile(filename):
        with open(filename) as json_file:
            json_data = json.load(json_file)
            return json_data

    headers = {'Content-type': 'application/json'}
    data = json.dumps(
            {
            "seriesid": [seriesid],
            "startyear": startyear, 
            "endyear": endyear,
            "registrationkey": os.environ['API_TOKEN_US_BUREAU_OF_LABOR_STATISTICS']
            }
        )
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)

    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        f.write(json.dumps(json_data))

    return json_data


get_bls_data('LNU02000000', '2022', '2023')


{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 97,
 'message': [],
 'Results': {'series': [{'seriesID': 'LNU02000000',
    'data': [{'year': '2023',
      'period': 'M12',
      'periodName': 'December',
      'value': '160754',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M11',
      'periodName': 'November',
      'value': '162149',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M10',
      'periodName': 'October',
      'value': '161676',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M09',
      'periodName': 'September',
      'value': '161669',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M08',
      'periodName': 'August',
      'value': '161427',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M07',
      'periodName': 'July',
      'value': '161982',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M06',
      'periodName': 'June',
      'value': '161559',
      'foot

In [11]:
from bls_gov.BlsGov import BlsGov

bls_gov = BlsGov()
bls_gov.get_data('LNU02000000', '2022', '2023')

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 97,
 'message': [],
 'Results': {'series': [{'seriesID': 'LNU02000000',
    'data': [{'year': '2023',
      'period': 'M12',
      'periodName': 'December',
      'value': '160754',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M11',
      'periodName': 'November',
      'value': '162149',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M10',
      'periodName': 'October',
      'value': '161676',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M09',
      'periodName': 'September',
      'value': '161669',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M08',
      'periodName': 'August',
      'value': '161427',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M07',
      'periodName': 'July',
      'value': '161982',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M06',
      'periodName': 'June',
      'value': '161559',
      'foot